# Preprocessing 
# Review data


## Formating of review data


In [43]:
import pandas as pd


file_path = "/Users/apple/Documents/GitHub/Steam-Market-Data-ML/very_final_reviews_copy.csv"
df = pd.read_csv(file_path)

# Convert 'game_time' to a numeric value
# not using regular expression regex=False hence treat as regular string
# The astype() method in Python, particularly within the Pandas library, is used for explicitly converting the data type of a Series or DataFrame column. 
# It enables you to change the data type of one or more columns, which is crucial for data manipulation and analysis.
df['game_time'] = df['game_time'].str.replace(" hrs on record", "", regex=False)
df['game_time'] = df['game_time'].str.replace(",", "", regex=False).astype(float)

# Convert 'helpful_count' to integer



# Extract the first number for helpful_count (the number of people who found the review helpful)

# extract first number
# https://www.reddit.com/r/learnprogramming/comments/vk0ujs/in_regular_expressions_what_is_the_difference/
# https://www.reddit.com/r/learnprogramming/comments/vk0ujs/in_regular_expressions_what_is_the_difference/

df['helpful_count'] = df['helpful_count'].str.extract(r'(\d+)')

# Fill NaN values with 0 (to handle cases where no number is found)
df['helpful_count'] = df['helpful_count'].fillna(0)
df['helpful_count'] = df['helpful_count'].replace("No one has rated this review as helpful yet", "0")
# Extract the second number for award_count (if present)



df['helpful_count'] = df['helpful_count'].astype(int)

# Print the cleaned dataframe
print(df['helpful_count'])

# Remove the "Posted: " prefix
df['review_date'] = df['review_date'].str.replace("Posted: ", "", regex=False)

# Convert the 'review_date' column into datetime format (allow pandas to infer the format)
# https://dateutil.readthedocs.io/en/stable/parser.html convinently data time uses https://dateutil.readthedocs.io/en/stable/parser.html which automatically
# recognize date name!
# https://medium.com/@mayurdalvi.5/100-days-of-data-science-and-machine-learning-day-6-date-and-time-manipulation-with-pandas-03abb8f48ba4
# I am not clear 100% how the strings are recognized

# (?<!2024): Negative lookbehind, meaning "do not match if '2024' is right before this position."
# replace the end with ", 2024"
#https://www.tpointtech.com/regex-lookbehind-in-python negative lookbehind
df['review_date'] = df['review_date'].str.replace(r"(?<!2024)$", ", 2024", regex=True)
df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce')

# Print the cleaned 'review_date' column

output_file_path = "/Users/apple/Documents/GitHub/Steam-Market-Data-ML/cleaned_reviews.csv"
df.to_csv(output_file_path, index=False)


0        721
1        518
2        535
3        293
4        301
        ... 
83517      1
83518      2
83519      0
83520      0
83521     27
Name: helpful_count, Length: 83522, dtype: int64


# Create percentage golden data

In [ ]:
#  Create a dataset that is by a certain percentage unlabled

import pandas as pd
import numpy as np 
unlabeled_df = pd.read_csv("/Users/apple/Documents/GitHub/Steam-Market-Data-ML/cleaned_reviews.csv")
print(len(unlabeled_df))
unlabel_percentage = 0.3
num_unlabeled = int(len(df) * unlabel_percentage)

# https://numpy.org/doc/2.1/reference/random/generated/numpy.random.choice.html
np.random.seed(2025)

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html all the attributes
# https://numpy.org/doc/2.1/reference/random/generated/numpy.random.choice.html
# replace=False meaning that value can only be selected once
unlabeled_df.index
unlabeled_index = np.random.choice(unlabeled_df.index, num_unlabeled, replace=False)
print(type(unlabeled_index))
print(unlabeled_index)


unlabeled_df.loc[unlabeled_index, "recommend"] = np.nan  # Set selected labels to NaN (unlabeled)

unlabeled_df.to_csv("unlabeled_reviews.csv", index=False)



print(unlabeled_df.isna().sum())





83522
<class 'numpy.ndarray'>
[62826  8158 41253 ... 33412  4496 63762]
steam_id              0
game_time             0
recommend         25056
helpful_count         0
review_date           0
review_comment        1
early_access          0
refund                0
game_id               0
name                  0
dtype: int64


This is used to simulate smaller sample

# Combining and fixing the data for player count and macro data

In [85]:


import pandas as pd


player_count_path = "/Users/apple/Documents/GitHub/Steam-Market-Data-ML/player_count_reviews.csv"
player_count = pd.read_csv(player_count_path)


player_count["player_count"] =  player_count["player_count"].str.replace(",", "", regex=False).astype(float)



player_count["gain"] = player_count["gain"].str.replace(",", "", regex=False).astype(float)

player_count["date"] = pd.to_datetime(player_count["date"], errors='coerce')


 
macro_data_path = "/Users/apple/Documents/GitHub/Steam-Market-Data-ML/macro_reviews.csv"
macro_data = pd.read_csv(macro_data_path)

merge_data = pd.merge(player_count,macro_data, on="game_id",how="left")

merge_data["total_review"] = merge_data["total_review"].str.replace(",", "", regex=False).astype(float)
merge_data["peak_player"] = merge_data["peak_player"].str.replace(",", "", regex=False).astype(float)
merge_data["game_id"] = merge_data["game_id"].astype(int)
merge_data['positive_percent'] = merge_data['positive_percent'].str.replace('%', '').astype(float) / 100
merge_data.to_csv("merged_player_count.csv", index=False)


/var/folders/nz/1d138x554532lzm46blcd3y80000gn/T/ipykernel_36252/3223595577.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  player_count["date"] = pd.to_datetime(player_count["date"], errors='coerce')
